# Parralellism in Clojure

###### REFS:
https://www.braveclojure.com/concurrency/  
https://www.technologyreview.com/s/421186/why-cpus-arent-getting-any-faster/  
https://www.allprogrammingtutorials.com/tutorials/task-parallelism-vs-data-parallelism.php  
https://clojuredocs.org/clojure.core/pmap

# Before we begin:

I found a very useful reference for strange characters in Clojure and recommend people save it.

https://clojure.org/guides/weird_characters

## Part 1: General Concurrency

Concurrent computing refers to the form of computing where multiple computations are made at the same time (in parallel) in a system instead of sequentially. Each computation must have its own point of execution for a system to be trully concurrent. In other words, you require a processor for every process.

CPU speeds are not increasing at the same rate as they used to. This is due to higher speeds requiring more power exponentially. More power -> more heat
As a result, chip manufactures have been focusing on adding more cores.

Which means....
Parallelism is going to become increasingly important moving forward.


### Advantages
<img src="https://vignette.wikia.nocookie.net/meme/images/4/42/1385136139955.png/revision/latest?cb=20150207013804" alt="drawing" width="100" align="left"/>

# Your program can be much faster
T(parallel) = (T(serial)/ IndependentNumberOfProcesses) + T(overhead)

### Disadvantages
Parallel programs can be much more complex to write and much more complex to debug.

### Pit Falls

#### Deadlock
    Multiple threads are wating for the same resource and the system comes to a halt.
        e.g. Dining Philosophers
    
#### Reference Cell Problem
    Multiple threads are reading/writing to the same location and memory and as a result the value depends on the execution order of the threads.

#### Mutual Exclusion, a recipe for disaster
    Multiple threads are reading/writing multiple instructions to a data source and as a result the data becomes jumbled.
    
    Imagine 2 threads trying to write recipes for to a file.
        The first states:
            "Boil Water in pot"
            "Add egg to pot"
            "Wait 3 minutes"
            "Remove Egg"
        The second states:
            "Mix Flour and sugar in pot"
            "Mix in Milk"
            "Put in microwave for 5 minutes"
            
    If the file does not enforce mutual exclusion you can potentially end up with:
            "Boil Water in a pot"
            "Mix Flour and sugar in pot"
            "Mix in milk"
            "Add egg to pot"
            "Put in microwave for 5 minutes"
            "Wait 3 minutes"
            "Remove Egg"
            
## Part 2: Futures

Clojure has a really simple way of implementing parallel programming.
These are called futures:

In [7]:
(future (Thread/sleep 1000) (println "wow"))
(println "I can print because I'm on a different thread!")

clojure.core$future_call$reify__8097@161f2089

future will place the corresponding code onto another thread as soon as possible and then continue running its own program.
This is why the "I can print..." message will print before "wow" even if Thread/sleep is called.

### References

futures are very much "set and forget".... unless you require a result back from them.

This is achieved by giving the future a reference tag that we can refer to later.

In [ ]:
(let [reference (future (Thread/sleep 1000) (+ 2 2))]
    (println @reference) ; will not execute until sleep is complete
    (println (deref reference)))

In this snippit we are assigning the tag "reference" to the (future ...) statement.
(println @reference) will not execute until the future is completed.

"deref" is synonymous with "@"

### Multiple Dereferences

When you run a function with future it will only run once. Any subsequent calls to the reference will return a cached value of its orignal result.

In [ ]:
(let [reference (future (println "This will only print once"))]
    (println @reference)
    (println (deref reference)))

### Time limits

You can also set a time limit using deref:

In [ ]:
; Here the thread is made to wait past the limit of 10ms and thus returns 5
(deref (future (Thread/sleep 6000) (+ 2 2)) 10, 5)
; Here the thread executes immediately and returns the result of the future
(deref (future (+ 2 2)) 5, 5)

### Realize?

If you want to check if a future is done computing you can use "realize?"

In [ ]:
(let [reference (future (Thread/sleep 1000) (+ 2 2))]
    (println (realized? reference))) ; This will return false since the thread is sleeping

(let [reference (future (+ 2 2))]
    (println (realized? reference))) ; This will return true since (+ 2 2) is executed immediately

Something to make note of is that futures will not return exceptions until dereferenced. For example:

In [ ]:
(def sequentialSub (-)) ; will break immediately

; vs

(def futureSub (future (-))) ; will not err until dereferenced
(@futureSub)

## Part 3: Delays

Delays allow you to define a task without having to run it immediately. Calling the delay's reference will cause the thread to block until it's execution is complete.

In [ ]:
(def runWhenHungry
    (delay (println "You acquire a tin of beans!")))

Multiple dereferences will return nil... but what if I get hungry again in a bit?

In [ ]:
(def runWhenHungry
    (delay (let [message "You eat some beans"]
               (println "You acquire a tin of beans!")
               `(println ~message)))) ; Tried to do it with quoting so it would run but I 
                                      ; didn't manage it

(def runWhenHungry
    (delay (let [message "You eat some beans"]
               (println "You acquire a tin of beans!")
               `(println ~message))))

@runWhenHungry

This is useful for executing a task once a future is completed.
For example:

In [ ]:
(def menu ["Fries" "Burger" "Drink"]) ;  A list of items on the menu

(defn callOrderNumber 
  [orderNumber]
  (println "Calling order: " orderNumber))

(defn deliverItem
  [item]
  (println item " is ready for pickup")
  true)

(let [notify (delay (callOrderNumber 1337))]
  (doseq [item menu]
    (future (deliverItem item)
            (force notify)))) ; Force is the same as deref but it is more obvious

We do not want to call the order number every time an item is ready, by using delay it is called once the first item is ready.

## Part 4: Promises

A promise is like declaring a variable that you promise to define in the future. You do not have to define when or how that value is produced. You can then "Deliver" a result to that promise and dereference it. For example:

In [ ]:
(def theAnswerToLifeTheUniverseAndEverything (promise))
;many million years later
(deliver theAnswerToLifeTheUniverseAndEverything (42))

@theAnswerToLifeTheUniverseAndEverything
; => 42

You can only deliver on a promise once.

If you try to dereference a promise before you've delivered a value the thread will block indefinitely (and the computer will lose its faith in your promises)

As it stands this doesn't seem all that useful but it can be powerful when coupled with future to search for values in a list.

This example from braveclojure does a great job at showing the power of promises.

Suppose, for some reason, that you're interested in buying the smoothest yak butter on the market for under $100. You're too lazy to actually look through websites so you create a web scraper to find online yak-store URLs.

In [8]:
(def yak-butter-international
  {:store "Yak Butter International"
    :price 90
    :smoothness 90})

(def butter-than-nothing
  {:store "Butter Than Nothing"
   :price 150
   :smoothness 83})

;; This is the butter that meets our requirements
(def baby-got-yak
  {:store "Baby Got Yak"
   :price 94
   :smoothness 99})

(defn mock-api-call
  [result]
  (Thread/sleep 1000)  ; This is to simulate an actual API call
  result)

(defn satisfactory?
  "If the butter meets our criteria, return the butter, else return false"
  [butter]
  (and (<= (:price butter) 100)
       (>= (:smoothness butter) 97)
       butter))

#'beaker_clojure_shell_bc50c7e7-4133-42ba-8846-d416269c693d/satisfactory?

Sequentially we would have to call "satisfactory?" on each of the api calls to determine if such an elusive butter exists, as the following code shows:

In [9]:
(time (some (comp satisfactory? mock-api-call)
            [yak-butter-international butter-than-nothing baby-got-yak]))
;"Elapsed time: 3000.7806 msecs"
;=> {:store "Baby Got Yak", :price 94, :smoothness 99}

"Elapsed time: 3003.6574 msecs"


For reference, "comp" combines multiple functions into one and applies them to the parameters from right to left. 
More info here:
https://clojuredocs.org/clojure.core/comp

Ok brace yourselves, heres the same in parallel using promise/future

In [ ]:
(time
 (let [butter-promise (promise)]
   (doseq [butter [yak-butter-international butter-than-nothing baby-got-yak]]
     (future (if-let [satisfactory-butter (satisfactory? (mock-api-call butter))]
               (deliver butter-promise satisfactory-butter))))
   (println "And the winner is:" @butter-promise)))
; => "Elapsed time: 1002.652 msecs"
; => And the winner is: {:store Baby Got Yak, :smoothness 99, :price 94}

Let's break it down!

```Clojure
(time
; times the function
 (let [butter-promise (promise)]
     ; declares a promised called butter-promise
   (doseq [butter [yak-butter-international butter-than-nothing baby-got-yak]]
       ; simple do-seq, will iterate through all the stores
     (future (if-let [satisfactory-butter (satisfactory? (mock-api-call butter))]
                 ; For each store in the list, create a new thread which determines
                 ; if the result of the api-call on the store is satisfactory
               (deliver butter-promise satisfactory-butter))))
                 ; if it IS satisfactory, deliver the store information to the
                 ; "butter-promise" promise
   (println "And the winner is:" @butter-promise)))
    ; prints the promise
```

There is one logical issue with this function. Can somebody guess what it is?

If you said the thread will lock if no satisfactory butter is found then you are correct!

Now how do we fix it??

If you recall I previously explained that we can set a time-out parameter when calling deref/"@". So if we change:

```Clojure
(println "And the winner is:" @butter-promise)))

; to

(println "And the winner is:" (deref butter-promise 3000 "No Butter Found"))))
```
And just like that, the problem is fixed. Except this is not really elegant since we're relying on time. What happens if we find more stores?

### Promises as callbacks
Promises can be used as JS-like callbacks in conjuction with future like so:

In [ ]:
(let [waterBoilingTemp (promise)]
  (future (println "Water boils at:" @waterBoilingTemp))
  (Thread/sleep 4000)
  (deliver waterBoilingTemp "100 celsius") nil)

Since the print statement is on another thread it will block until the promise is fulfilled 4 seconds later when we deliver it.

## Part 5: Some clojure macros/functions that are parallel

- binding
- pmap
- pcalls
- pvalues
- with-redefs

### binding

```Clojure
(binding bindings & body)
```
Works the same as let but generates the bindings in parallel and lets you bind function calls.



In [ ]:
(defn mymax [x y]
  (min x y))

(defn find-max [x y]
  (max x y))

(let [max mymax]
         (find-max 10 20))
;=> 20
;max is not being rebinded to mymax

user=> (binding [max mymax]
         (find-max 10 20))

;=> 10
;max is being rebinded to mymax

### pmap

```Clojure
(pmap f coll) (pmap f coll & colls)
```

The same as map BUT it applies the function in parallel.


In [10]:
(pmap inc [1 2 3 4 5])

[2, 3, 4, 5, 6]

### pcalls

```Clojure
(pcalls & fns)
```

Executes all the no argument fns in parallel

In [12]:
(pcalls (println "Wow") (println "this") (println "is") (println "in") (println "parallel"))

Wow
this
is
in
parallel


Object:  class clojure.lang.LazySeq, value cannot be displayed due to following error

### pvalues

```Clojure
(pvalues & exprs)
```

Returns a lazy sequence of computationally heavy expression

### with-redefs

```Clojure
(with-redefs bindings & body)
```

Runs a body using bindings made in parallel

## Practice!

Q1: Create a countdown that calls a new future for every second of the countdown.
Q2: 

Bonus Assignmnet: Help me figure out when to use atoms